In [143]:
from logging import raiseExceptions
from tokenize import Double
import networkx as nx
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import meshplot as mp
import torch
torch.manual_seed(125)
import random
random.seed(125)
from modelovae import Node

In [144]:
use_gpu = True
device = torch.device("cuda:0" if use_gpu and torch.cuda.is_available() else "cpu")

def count_fn(f):
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        return f(*args, **kwargs)
    wrapper.count = 0
    return wrapper

In [145]:
def plotTree( root, dec ):
    graph = nx.Graph()
    root.toGraph( graph, 0, dec)
    edges=nx.get_edge_attributes(graph,'procesada')
   
    p = mp.plot( np.array([ graph.nodes[v]['posicion'] for v in graph.nodes]), shading={'point_size':1}, return_plot=True)
    print("nodes", graph.nodes)
    for arista in graph.edges:
        p.add_lines( graph.nodes[arista[0]]['posicion'], graph.nodes[arista[1]]['posicion'], shading={'color':'red'})

    return 

def subplotTree( root, dec, d, c = "black"):
    graph = nx.Graph()
    root.toGraph( graph, 0, dec)
    edges=nx.get_edge_attributes(graph,'procesada')

    shading={'point_size':0.1, "point_color": c, "line_color": 'black'}
    #d = mp.plot( np.array([ graph.nodes[v]['posicion'] for v in graph.nodes]), shading={'point_size':0.1}, return_plot=True)
    d.add_points( np.array([ graph.nodes[v]['posicion'] for v in graph.nodes]), shading = shading)
    for arista in graph.edges:
        d.add_lines( graph.nodes[arista[0]]['posicion'], graph.nodes[arista[1]]['posicion'], shading  = shading)


    return d

def traverse(root, tree):
       
        if root is not None:
            traverse(root.left, tree)
            tree.append((root.radius, root.data))
            traverse(root.right, tree)
            return tree

def traversem(root):
       
        if root is not None:
            traversem(root.left)
            print(root)
            traversem(root.right)
            return 

def traverse_2(tree1, tree2, t_l):
       
        if tree1 is not None:
            traverse_2(tree1.left, tree2.left, t_l)
            if tree2:
                t_l.append((tree1.radius, tree2.radius))
                print((tree1.radius, tree2.radius))
            else:
                t_l.append(tree1.radius)
                print((tree1.radius))
            traverse_2(tree1.right, tree2, t_l)
            return t_l
            

def traverse_conexiones(root, tree):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            traverse_conexiones(root.left, tree)
            if root.right is not None:
                tree.append((root.data, root.right.data))
            if root.left is not None:
                tree.append((root.data, root.left.data))
            traverse_conexiones(root.right, tree)
            return tree


@count_fn
def createNode(data, radius, left = None, right = None):
        """
        Utility function to create a node.
        """
        return Node(data, radius, left, right)
 
def deserialize(data):
    if  not data:
        return 
    nodes = data.split(';')  
    #print("node",nodes[3])
    def post_order(nodes):
                
        if nodes[-1] == '#':
            nodes.pop()
            return None
        node = nodes.pop().split('_')
        data = int(node[0])
        #radius = float(node[1])
        #print("node", node)
        #breakpoint()
        radius = node[1]
        #print("radius", radius)
        rad = radius.split(",")
        rad [0] = rad[0].replace('[','')
        rad [3] = rad[3].replace(']','')
        r = []
        for value in rad:
            r.append(float(value))
        #r =[float(num) for num in radius if num.isdigit()]
        r = torch.tensor(r, device=device)
        #breakpoint()
        root = createNode(data, r)
        root.right = post_order(nodes)
        root.left = post_order(nodes)
        
        return root    
    return post_order(nodes)    


def read_tree(filename, dir):
    with open('./' +dir +'/' +filename, "r") as f:
        byte = f.read() 
        return byte

In [146]:
def numerar_nodos(root, count):
    if root is not None:
        numerar_nodos(root.left, count)
        root.data = len(count)
        count.append(1)
        numerar_nodos(root.right, count)
        return count


def traversefeatures(root, features):
       
    if root is not None:
        traversefeatures(root.left, features)
        features.append(root.radius)
        traversefeatures(root.right, features)
        return features



def traversefeatures(root, features):
       
    if root is not None:
        traversefeatures(root.left, features)
        features.append(root.radius)
        traversefeatures(root.right, features)
        return features

def calcularLossAtributo(nodo, radio):
    if nodo is None:
        return
    radio = radio.reshape(-1,4)
    nodo = nodo.radius.reshape(-1,4)
    l2    = nn.MSELoss()
   
    mse = l2(radio, nodo)
    return mse

def searchNode(node, key):
     
    if (node == None):
        return False
 
    if (node.data == key):
        return node
        
 
    """ then recur on left subtree """
    res1 = searchNode(node.left, key)
    # node found, no need to look further
    if res1:
        return res1
 
    """ node is not found in left,
    so recur on right subtree """
    res2 = searchNode(node.right, key)
    return res2

def getLevelUtil(node, data, level):
    if (node == None):
        return 0
 
    if (node.data == data):
        return level
 
    downlevel = getLevelUtil(node.left, data, level + 1)

    if (downlevel != 0):
        return downlevel
 
    downlevel = getLevelUtil(node.right, data, level + 1)
    return downlevel
 
# Returns level of given data value
 
 
def getLevel(node, data):
    return getLevelUtil(node, data, 1)
 

def setLevel(data_loader):
    for d in data_loader:
        for data in d:
            max_level = 0
            tree = list(data.keys())[0]
            n_nodes = data[tree]#[0]
            count = []
            numerar_nodos(tree, count)
            for x in range(0, n_nodes):
                level = getLevel(tree, x)
                if level > max_level:
                    max_level = level
                if (level):
                    node = searchNode(tree, x)
                    node.level = getLevel(tree, x)
                else:
                    print(x, "is not present in tree")
            tree_level = []
            tree.getTreeLevel(tree, tree_level)
            tree.setTreeLevel(tree, sum(tree_level))
            tree.setMaxLevel(tree, max_level)

def calcularLossEstructura(cl_p, original, mult):
        
        if original is None:
            return
        ce = nn.CrossEntropyLoss(weight = mult)

        if original.childs() == 0:
            vector = [1, 0, 0] 
        if original.childs() == 1:
            vector = [0, 1, 0]
        if original.childs() == 2:
            vector = [0, 0, 1] 


        c = ce(cl_p, torch.tensor(vector, device=device, dtype = torch.float).reshape(1, 3))
        return c


In [147]:
def norm(root, minx, miny, minz, minr, maxx, maxy, maxz, maxr):
    
    if root is not None:
        mx = minx.clone().detach()
        my = miny.clone().detach()
        mz = minz.clone().detach()
        mr = minr.clone().detach()
        Mx = maxx.clone().detach()
        My = maxy.clone().detach()
        Mz = maxz.clone().detach()
        Mr = maxr.clone().detach()
       
        root.radius[0] = (root.radius[0] - minx)/(maxx - minx)
        root.radius[1] = (root.radius[1] - miny)/(maxy - miny)
        root.radius[2] = (root.radius[2] - minz)/(maxz - minz)
        root.radius[3] = (root.radius[3] - minr)/(maxr - minr)
        
        norm(root.left, mx, my, mz, mr, Mx, My, Mz, Mr)
        norm(root.right, mx, my, mz, mr, Mx, My, Mz, Mr)
        return 

def normTodos(root, minx, miny, minz, minr, maxx, maxy, maxz, maxr):
    
    if root is not None:
        mx = minx.clone().detach()
        my = miny.clone().detach()
        mz = minz.clone().detach()
        mr = minr.clone().detach()
        Mx = maxx.clone().detach()
        My = maxy.clone().detach()
        Mz = maxz.clone().detach()
        Mr = maxr.clone().detach()
       
        M = max((maxx - minx), (maxy - miny), (maxz - minz))
        root.radius[0] = (root.radius[0] - minx)/M
        root.radius[1] = (root.radius[1] - miny)/M
        root.radius[2] = (root.radius[2] - minz)/M
        root.radius[3] = (root.radius[3])/M
        
        normTodos(root.left, mx, my, mz, mr, Mx, My, Mz, Mr)
        normTodos(root.right, mx, my, mz, mr, Mx, My, Mz, Mr)
        return 

def normalize_features(root):
    features = []
    features = traversefeatures(root, features)
    
    x = [tensor[0] for tensor in features]
    y = [tensor[1] for tensor in features]
    z = [tensor[2] for tensor in features]
    r = [tensor[3] for tensor in features]
 
    normTodos(root, min(x), min(y), min(z), min(r), max(x), max(y), max(z), max(r))

    return 

In [148]:
def normalizar(root, M, desplazamiento_x, desplazamiento_y, desplazamiento_z):
    
    if root is not None:
        '''
        mx = minx.clone().detach()
        my = miny.clone().detach()
        mz = minz.clone().detach()
        mr = minr.clone().detach()
        Mx = maxx.clone().detach()
        My = maxy.clone().detach()
        Mz = maxz.clone().detach()
        Mr = maxr.clone().detach()
       '''
        
        root.radius[0] = (root.radius[0] - desplazamiento_x)/M
        root.radius[1] = (root.radius[1] - desplazamiento_y)/M
        root.radius[2] = (root.radius[2] - desplazamiento_z)/M
        root.radius[3] = (root.radius[3])/M
        normalizar(root.left, M, desplazamiento_x, desplazamiento_y, desplazamiento_z)
        normalizar(root.right, M, desplazamiento_x, desplazamiento_y, desplazamiento_z)
        return 

def centrarRoot(root):
    features = []
    features = traversefeatures(root, features)
    
    x = [tensor[0] for tensor in features]
    y = [tensor[1] for tensor in features]
    z = [tensor[2] for tensor in features]
    r = [tensor[3] for tensor in features]
    
    M = max((max(x) - min(x)), (max(y) - min(y)), (max(z) - min(z)))
    desplazamiento_x = root.radius.cpu().numpy()[0]
    desplazamiento_y = root.radius.cpu().numpy()[1]
    desplazamiento_z = root.radius.cpu().numpy()[2]

    normalizar(root, M, desplazamiento_x, desplazamiento_y, desplazamiento_z)

    return

In [149]:
def my_collate(batch):
    return batch


class tDataset(Dataset):
    def __init__(self, l, dir, transform=None):
        self.names = l
        self.transform = transform
        self.data = [] #lista con las strings de todos los arboles
        for file in self.names:
            self.data.append(read_tree(file, dir))
        self.trees = []
        self.Trees = {}
        for tree in self.data:
            deserial = deserialize(tree)
            #centrarRoot(deserial)
            normalize_features(deserial)
            c = []
            n = deserial.countNodes(deserial, c)
            self.trees.append(deserial)
            

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        tree = self.trees[idx]
        name = self.names[idx]
        return tree, name

batch_size = 1

In [150]:
import uuid
from ipywidgets import Output, HBox
rendertype = 'JUPYTER'

class Subplot():
    def __init__(self, data, view, s):
        if data == None:
            self.rows = []
            self.hboxes = []
        else:
            self.rows = data.rows
        if s[0] != 1 or s[1] != 1:
            if data == None: # Intialize subplot array
                cnt = 0
                for r in range(s[0]):
                    row = []
                    for c in range(s[1]):
                        row.append(Output())
                        cnt += 1
                    self.rows.append(row)

                for r in self.rows:
                    hbox = HBox(r)
                    if rendertype == "JUPYTER":
                        display(hbox)
                    self.hboxes.append(hbox)
        
            out = self.rows[int(s[2]/s[1])][s[2]%s[1]]
            if rendertype == "JUPYTER":
                with out:
                    display(view._renderer)
            self.rows[int(s[2]/s[1])][s[2]%s[1]] = view

    def save(self, filename=""):
        if filename == "":
            uid = str(uuid.uuid4()) + ".html"
        else:
            filename = filename.replace(".html", "")
            uid = filename + '.html'

        s = ""
        imports = True
        for r in self.rows:
            for v in r:
                s1 = v.to_html(imports=imports, html_frame=False)
                s = s + s1
                imports = False

        s = "<html>\n<body>\n" + s + "\n</body>\n</html>"
        with open(uid, "w") as f:
            f.write(s)
        print("Plot saved to file %s."%uid)

    def to_html(self, imports=True, html_frame=True):
        s = ""
        for r in self.rows:
            for v in r:
                s1 = v.to_html(imports=imports, html_frame=html_frame)
                s = s + s1
                imports = False

        return s

In [151]:
def subplot(f, c = 'red', uv=None, n=None, shading={}, s=[1, 1, 0], data=None, **kwargs):
    
    shading={'point_size':0.1, "point_color": c, "line_color": c, "width":400, "height":400}
    view = mp.Viewer(settings = {"width": 500, "height": 500, "antialias": True, "scale": 1.5, "background": "#ffffff",
                "fov": 30})

    obj = view.add_points(np.array([ f.nodes[v]['posicion'] for v in f.nodes]), shading=shading)
    #view.add_lines(v, f, shading=shading)
    for arista in f.edges:
        obj = view.add_lines( f.nodes[arista[0]]['posicion'], f.nodes[arista[1]]['posicion'], shading  = shading)
   
    subplot = Subplot(data, view, s)
    #subplot = view
    return subplot

def sTree( root, dec, s, c, d=None):
    graph = nx.Graph()
    root.toGraph( graph, 0, dec)
    #edges=nx.get_edge_attributes(graph,'procesada')

    #shading={'point_size':0.1, "point_color": c, "line_color": c}
    #d = subplot( graph, c, s)
    if d:
        subplot(graph, c=c, s=s, data = d)
    else:
        
        d = subplot(graph, c=c, s=s)
    
    #for arista in graph.edges:
    #    d.add_lines( graph.nodes[arista[0]]['posicion'], graph.nodes[arista[1]]['posicion'], shading  = shading)
    return d

In [152]:
def remove_below_level(node, level, current_level=1):
    if not node:
        return None
    
    node.left = remove_below_level(node.left, level, current_level + 1)
    node.right = remove_below_level(node.right, level, current_level + 1)
    
    if current_level > level:
        return None
    node.radius = node.radius.tolist()
    return node


In [153]:
def removebelowlevel(node, level, current_level=1):
    #para dibujarlos, no guardarlos
    if not node:
        return None
    
    node.left = removebelowlevel(node.left, level, current_level + 1)
    node.right = removebelowlevel(node.right, level, current_level + 1)
    
    if current_level > level:
        return None
    return node


In [154]:
#t_list = os.listdir("../arbolesLimpios/Trees/" )
#dataset = tDataset(t_list, "../arbolesLimpios/Trees")
t_list = os.listdir("../arboles/sinteticos-10/" )
dataset = tDataset(t_list, "../arboles/sinteticos-10/")
data_loader = DataLoader(dataset, batch_size = batch_size, shuffle=False, collate_fn=my_collate)


i = 0
d = None
n = 10

for batch in data_loader:
    for input in batch:
        name = input[1]
        input = input[0]
        '''
        if d is None:
            d = sTree(input, False, c = "red", s = [20,2,i])
        else:
            sTree(input, False, c = "red", s = [20,2,i], d=d)
        '''
        decoded = remove_below_level(input, n)
        #sTree(decoded, False,  s = [20,2,i+1], c = "black", d=d)
        serial = decoded.serialize(decoded)
        file = open("../arbolesLimpios/sinteticos10/s10" + name , "w")
        file.write(serial)
        file.close() 
        i += 1


In [155]:
p

NameError: name 'p' is not defined

Cortar por cantidad de nodos

In [156]:
def remove_after_n_nodes(root, n):

    if not root:
        return None
    
    count = 0
    queue = [root]
    
    while queue:
        node = queue.pop(0)
        count += 1
        
        if count > n:
            return None
        
        if node.left:
            queue.append(node.left)
            
        if node.right:
            queue.append(node.right)
            
    return root


In [157]:
def copy_first_n_nodes(node, n):
    ##para guardar
    if not node or n == 0:
        return None
    
    # Create a new node with the same data as the current node
    new_node = Node(node.data, node.radius.tolist())
    
    # Recursively copy the left and right subtrees up to n-1 nodes
    if n > 1:
        new_node.right = copy_first_n_nodes(node.right, n-1)
        if new_node.right is None and n > 2:
            # If there are less than n-1 nodes in the left subtree, copy nodes from the right subtree
            new_node.left = copy_first_n_nodes(node.left, n-2)
    elif n == 1:
        new_node.left = None
        new_node.right = None
    
    # Return the new node
    return new_node


def count_nodes(node):
    if not node:
        return 0
    
    return count_nodes(node.left) + count_nodes(node.right) + 1


In [158]:
def copy_first_n_nodes(node, n):
    ##para dibujar
    if not node or n == 0:
        return None
    
    # Create a new node with the same data as the current node
    new_node = Node(node.data, node.radius)
    
    # Recursively copy the left and right subtrees up to n-1 nodes
    if n > 1:
        new_node.right = copy_first_n_nodes(node.right, n-1)
        if new_node.right is None and n > 2:
            # If there are less than n-1 nodes in the left subtree, copy nodes from the right subtree
            new_node.left = copy_first_n_nodes(node.left, n-2)
    elif n == 1:
        new_node.left = None
        new_node.right = None
    
    # Return the new node
    return new_node


def count_nodes(node):
    if not node:
        return 0
    
    return count_nodes(node.left) + count_nodes(node.right) + 1

In [159]:
t_list = os.listdir("arbolesLimpios/Trees/" )[:10] 
dataset = tDataset(t_list)
data_loader = DataLoader(dataset, batch_size = batch_size, shuffle=True, collate_fn=my_collate)


set_tree_level(data_loader)

i = 0
d = None
n = 20

for batch in data_loader:
    for input in batch:
        '''
        if d is None:
            d = sTree(input, False, c = "red", s = [10,2,i])
        else:
            sTree(input, False, c = "red", s = [10,2,i], d=d)
       '''
        decoded = copy_first_n_nodes(input, n)
        #decoded.traverseInorder(decoded)
        print("n nodes", count_nodes(decoded), str(i) +".dat")
        #sTree(decoded, False,  s = [10,2,i+1], c = "black", d=d)
        serial = decoded.serialize(decoded)
        file = open("./n" +str(n) +"/" + str(i) +".dat", "w")
        file.write(serial)
        #file.close() 
        i += 2


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'arbolesLimpios/Trees/'